## Optiver Competition 
Team: Brandon, Jatindeep, Alex, Avi

**Overview:** I created a Jupyter notebook from our last notebook where we can develop models here and test the MAE since it takes hours on Kaggle. After we develop some solid models, we can then submit them on kaggle. We will automatically start with the 50 features, as well as the targets. And go from there. Feel free to add visualization as it will help us better understand the models responses to our changes.

Disregard Principal Component Analysis for now

## Principal Component Analysis
**PROBLEM: ALL CURRENT FEATURE VARS SCALED IN MULTIPLE DIFFERENT WAYS THUS PCA MIGHT NOT WORK**
- My idea is to transform the dataset of 50 feature vars and then feed it into whatever ensembling method we decide to use (right now it will be the one of the 3 models and averaged prediction)
- I will begin by plotting the cumulative explained variance

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Apply PCA to (already scaled) features with the desired number of components
n_components = 15  # Choose the number of components
pca = PCA(n_components=n_components)
X_pca = pca.fit_transform(df_) 

#Calculate explained and cumulative variance
explained_variance = pca.explained_variance_ratio_
cumulative_variance = explained_variance.cumsum()

# Plot the explained variance
plt.plot(range(1, n_components + 1), cumulative_variance, marker='o', linestyle='--')
plt.xlabel('Number of Principal Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('Explained Variance vs. Number of Principal Components')
plt.show()

It seems like 5 is a pretty reasonable number of PCs to use so I will continue with this to see if it improves our model

In [ ]:
# Choose the 5 components and transform the old feature vars
n_components_selected = 5
pca_selected = PCA(n_components=n_components_selected)
X_pca_selected = pca_selected.fit_transform(df_) #This is what we will use as the new X (feature) data

#Need to split the data again since it is transformed (just the x values but just including y for simplicity)
if TRAINING:
    X = X_pca_selected
    Y = df_target['target'].values

    # Remove rows with missing target values
    valid_indices = np.isfinite(Y)
    X = X[valid_indices]
    Y = Y[valid_indices]

    # Split the data into training, validation, and test sets
    X_train, X_temp, Y_train, Y_temp = train_test_split(X, Y, test_size=0.2, random_state=42)
    X_valid, X_test, Y_valid, Y_test = train_test_split(X_temp, Y_temp, test_size=0.5, random_state=42)

models = []

#Train each model (no cross validation)
train(model_dict, 'lgb')
train(model_dict, 'xgb')
train(model_dict, 'cbt')


## New Features and Model Setup
- This is the code I pulled from the other notebook mixed with all of our stuff. The goal is to beat our first model MAE of 6.41
- I will first begin with just the new features
- First up is feature engineering
- I took out the validation set because for two horrible nights it was an absolute nightmare splitting them. Doing the kfold thing with temps ended up putting nans and infinite values in there? Absolutely screwing up my models. So I took the eval sets out and Ill just get the performance on the test set at the end.

In [13]:
import pandas as pd
import numpy as np
import catboost as cbt
import xgboost as xgb
import lightgbm as lgb
import gc
from itertools import combinations
import warnings
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
from warnings import simplefilter
import joblib
from tqdm import tqdm

#Filtering some warnings (we might wanna look at the performance one maybe)
warnings.filterwarnings("ignore")
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

#Folds and train/test triggers
is_train = True
is_infer = True

#Read data, calculate median, std_deviation of total bid + sizes, get target
train = pd.read_csv('train.csv')
median_sizes = train.groupby('stock_id')['bid_size'].median() + train.groupby('stock_id')['ask_size'].median()
std_sizes = train.groupby('stock_id')['bid_size'].std() + train.groupby('stock_id')['ask_size'].std()
train = train.dropna(subset=['target'])

#The author's feature engineering. Could work well we could potentially improve and combine from last notebook if there are any differences
def feat_eng(df):
    
    cols = [c for c in df.columns if c not in ['row_id', 'time_id']]
    df = df[cols]
    df['imbalance_ratio'] = df['imbalance_size'] / df['matched_size']
    df['bid_ask_volume_diff'] = df['ask_size'] - df['bid_size']
    df['mid_price'] = (df['ask_price'] + df['bid_price']) / 2
    df['bid_plus_ask_sizes'] = df['bid_size'] + df['ask_size']
    df['median_size'] = df['stock_id'].map(median_sizes.to_dict())
    df['std_size'] = df['stock_id'].map(std_sizes.to_dict())
    df['high_volume'] = np.where(df['bid_plus_ask_sizes'] > df['median_size'], 1, 0)
        
    prices = ['reference_price','far_price', 'near_price', 'ask_price', 'bid_price', 'wap']
    
    for c in combinations(prices, 2):
        df[f'{c[0]}_minus_{c[1]}'] = (df[f'{c[0]}'] - df[f'{c[1]}']).astype(np.float32)
        df[f'{c[0]}_{c[1]}_imb'] = df.eval(f'({c[0]}-{c[1]})/({c[0]}+{c[1]})')
    for c in combinations(prices, 3):
        max_ = df[list(c)].max(axis=1)
        min_ = df[list(c)].min(axis=1)
        mid_ = df[list(c)].sum(axis=1)-min_-max_
        df[f'{c[0]}_{c[1]}_{c[2]}_imb2'] = (max_-mid_)/(mid_-min_)
    df.drop(columns=['date_id',], inplace=True)
    gc.collect()
    
    print('Features Finished Calculating')
    return df


## Model Setup

In [60]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

# Note the X, y columns
X = feat_eng(train.drop(columns='target'))
y = train['target'].values

# Remove rows with missing target values
valid_indices = np.isfinite(y)
X = X[valid_indices]
y = y[valid_indices]

# Split the data into training and test
X_train, X_temp, Y_train, Y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, Y_val, Y_test = train_test_split(X_temp, Y_temp, test_size=0.5, random_state=42)


# Save indices
train_index = np.arange(len(X_train))
val_index = np.arange(len(X_val))

#Save the bounds of the target
y_min = np.min(y)
y_max = np.max(y)

# Set parameters for XGBoost (These were ours think about cvgrid search on individual model)
params_xgb = {
    'tree_method': 'hist',
    'objective': 'reg:absoluteerror',
    'n_estimators': 500,
    'n_jobs': -1
}

# Set parameters for CatBoost (These were ours think about cvgrid search on individual model)
params_cbt = {
    'objective': 'MAE',
    'iterations': 500,
    'use_best_model': True,
    'task_type': 'GPU'
}



# Set params for LGBM (these were by the author besides my callback)
params_lgb = {
    'learning_rate': 0.018,
    'max_depth': 9,
    'n_estimators': 600,
    'num_leaves': 440,
    'objective': 'mae',
    'random_state': 42,
    'reg_alpha': 0.01,
    'reg_lambda': 0.01,
    'n_jobs': -1
}


# Create a dictionary of models (from Jatindeeps idea)
model_dict = {
    'xgb': xgb.XGBRegressor(**params_xgb),
    'cbt': cbt.CatBoostRegressor(**params_cbt),
    'lgb': lgb.LGBMRegressor(**params_lgb)
}


#Specify Number of Folds
n_folds = 2


# List to store models
models = []

# Path to store models
model_path = "C:/Users/User/Desktop/Kaggle_Models_Optiver"

# Add the train model with cross-validation
def train_models_with_cv(model_dict, modelname='lgb'):
    if is_train:
        X_train_fold = X_train[train_index%n_folds!=i]
        Y_train_fold = Y_train[train_index%n_folds!=i]
        X_val_fold = X_val[val_index%n_folds==i]
        Y_val_fold = Y_val[val_index%n_folds==i]
        
        model = model_dict[modelname]

        if modelname == 'lgb':
            callbacks = [lgb.callback.early_stopping(100, first_metric_only=True, verbose=True, min_delta=.0001)]
            model.fit(X_train_fold, Y_train_fold, eval_set=[(X_val_fold, Y_val_fold)], eval_metric='mae', callbacks=callbacks) 
            model = model.best_iteration_  # best model for lgb saved here

        if modelname == 'xgb':
            callbacks = xgb.callback.EarlyStopping(rounds=5, metric_name="mae", save_best=True, min_delta=.0001)
            model.fit(X_train_fold, Y_train_fold, eval_set=[(X_val_fold, Y_val_fold)], callbacks=[callbacks], verbose=True)

        if modelname == 'cbt':
            model.fit(X_train_fold, Y_train_fold, eval_set=[(X_val_fold, Y_val_fold)], verbose=1, early_stopping_rounds=100)
            model = model.best_iteration_  # best model for cbt saved here

        models.append(model)
        joblib.dump(model, f'{model_path}/{modelname}_{i}.model')
    else:
        models.append(joblib.load(f'{model_path}/{modelname}_{i}.model'))
        
            

# Train the models
for i in range(n_fold):
    print('Begin Training Models')
    train_models_with_cv(model_dict, 'lgb')
    train_models_with_cv(model_dict, 'xgb')
    train_models_with_cv(model_dict, 'cbt')


Features Finished Calculating
Begin Training Models
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.196085 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16973
[LightGBM] [Info] Number of data points in the train set: 2095156, number of used features: 70
[LightGBM] [Info] Start training from score -0.060201
Training until validation scores don't improve for 100 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [War

XGBoostError: [23:30:16] C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\common\../data/gradient_index.h:94: Check failed: valid: Input data contains `inf` or a value too large, while `missing` is not set to `inf`

### Performance Results from Upgraded Model
- I will now test on the test set and calculate the average MAE amongst folds for each model separately, and then make an averaged prediction, add final transformations, and calculate MAE for our final model

In [ ]:
from sklearn.metrics import mean_absolute_error
import numpy as np

#Transformation function from author 
def zero_sum(prices, volumes):
    std_error = np.sqrt(volumes)
    step = np.sum(prices)/np.sum(std_error)
    out = prices-std_error*step
    return out
    
# Initialize lists to store MAE for each model and predictions
mae_scores = {model_name: [] for model_name in model_dict}
all_predictions = []

# Loop through the folds
for fold in range(n_fold):
    valid_indices = index % N_fold == fold
    X_test = X_test[valid_indices]
    Y_test = Y_test[valid_indices]

    # Initialize lists to store predictions for each model
    model_predictions = {model_name: [] for model_name in model_dict}

    # Loop through the models and predict for each model
    for model in models:
        model_name = model.__class__.__name__
        y_pred_test = model.predict(X_Test)
        mae = mean_absolute_error(Y_test, y_pred_test)
        mae_scores[model_name].append(mae)
        model_predictions[model_name] = y_pred_test

    # Calculate the average prediction for this fold
    fold_average_prediction = np.mean(list(model_predictions.values()), axis=0)
    all_predictions.append(fold_average_prediction)

# Calculate the average MAE for each model
average_mae_scores = {model_name: np.mean(scores) for model_name, scores in mae_scores.items()}

# Calculate averaged predictions for all models
all_model_prediction = np.mean([model.predict(X_test) for model in models], 0)

# Perform zero sum transformation
zero_summed_prediction = zero_sum(all_model_prediction, X_test['bid_size'] + X_test['ask_size'])

#Get the final prediction by clipping
final_transformed_prediction = np.clip(zero_summed_prediction, y_min, y_max)

#Calculate the MAE for this averaged prediction
final_transformed_prediction_mae = mean_absolute_error(Y_test, final_transformed_prediction)

# Print the results
for model_name, score in average_mae_scores.items():
    print(f"{model_name} - {n_fold} fold MAE: {score}")

print(f"Averaged and Final Transformed Prediction MAE: {final_transformed_prediction_mae}")